# Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.


Create a Jupyter Notebook called database_engineering.ipynb and use this to complete all of your Database Engineering work.

In [1]:
# Import dependencies
import pandas as pd
import csv
import numpy as np
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text, create_engine, MetaData, ForeignKey

Use Pandas to read your cleaned measurements and stations CSV data.

In [2]:
# Import measurement CSV
df_measure = pd.read_csv('clean_hawaii_measurements.csv')
df_measure.head()

,id,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.00,73


In [3]:
# Import Station CSV
df_stations= pd.read_csv('hawaii_stations.csv')
df_stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


Use the engine and connection string to create a database called hawaii.sqlite.

In [4]:
# Create Engine and Connect it
engine = create_engine("sqlite:///hawaii.sqlite")

conn = engine.connect()

Use declarative_base and create ORM classes for each table.


You will need a class for Measurement and for Station.
Make sure to define your primary keys.

In [5]:
# Set dec_base, primary/foreign keys, and classes for two items
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key = True)
    station = Column(String(25), ForeignKey('stations.station'))
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)
class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key = True)
    station = Column(String(20))
    name = Column(String(255))
    lat = Column(Float)
    lng = Column(Float)
    elevation = Column(Float)


Once you have your ORM classes defined, create the tables in the database using 'create_all'.

In [6]:
# Create database with the newly created classes inside of it
Base.metadata.create_all(engine)

In [7]:
# Bring in CSV files and change them from simple pandas to records-type pandas
df_clean_measure = pd.read_csv('clean_hawaii_measurements.csv')
df_clean_stations = pd.read_csv('hawaii_stations.csv')
weather_data = df_clean_measure.to_dict(orient='records')
station_data = df_clean_stations.to_dict(orient='records')
station_data[0]

{'station': 'USC00519397',
 'name': 'WAIKIKI 717.2, HI US',
 'latitude': 21.2716,
 'longitude': -157.8168,
 'elevation': 3.0}

In [8]:
# Create metadata item to be used in creating the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [9]:
# Create tables
tab_weather = sqlalchemy.Table('measurements', metadata, autoload=True)
tab_station = sqlalchemy.Table('stations', metadata, autoload = True)

In [10]:
# Insert the panda records into the tables, column by column
conn.execute(tab_weather.insert(), weather_data)
conn.execute(tab_station.insert(), station_data)

In [11]:
# Verify that the info has been inserted into the table.
conn.execute("select * from stations limit 10").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', None, None, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', None, None, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', None, None, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', None, None, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', None, None, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', None, None, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', None, None, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', None, None, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', None, None, 152.4)]

In [12]:
# Verify that the info has been inserted into the table.
conn.execute("select * from measurements limit 10").fetchall()

[(0, 'USC00519397', '2010-01-01', 0.08, 65),
 (1, 'USC00519397', '2010-01-02', 0.0, 63),
 (2, 'USC00519397', '2010-01-03', 0.0, 74),
 (3, 'USC00519397', '2010-01-04', 0.0, 76),
 (4, 'USC00519397', '2010-01-06', 0.0, 73),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64)]